In [1]:
import pandas as pd

# Reviews

In [34]:
reviews = pd.read_csv(r"D:\My-Projects\stonecap\data\processed\reviews\labelled_dataset\combined_reviews\combined_reviews.csv")

In [35]:
reviews = reviews.drop(['Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], axis=1)


In [38]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3715625 entries, 0 to 3715624
Data columns (total 7 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   business_id      object
 1   stars            int64 
 2   useful           int64 
 3   text             object
 4   date             object
 5   sentiment        object
 6   predicted_label  object
dtypes: int64(2), object(5)
memory usage: 198.4+ MB


In [39]:
# fix the minimum 'useful' rating to 0
reviews.loc[reviews['useful'] == reviews['useful'].min(), 'useful'] = 0

In [41]:
reviews = reviews.rename(columns={'date': 'datetime'})
reviews['datetime'] = pd.to_datetime(reviews['datetime'])

In [9]:
# Saving review text information into another dataset.
review_text_data = reviews[['business_id', 'predicted_label', 'text', 'sentiment']].copy()

review_text_data.to_csv(r"D:\My-Projects\stonecap\data\processed\reviews\labelled_dataset\combined_reviews\reviews_textual_data.csv", index=False)

In [31]:
reviews['year'] = reviews['datetime'].dt.year
reviews_per_year = reviews.groupby(['business_id', 'year']).count().reset_index()[['business_id', 'year', 'stars']]
reviews_per_year = reviews_per_year.rename(columns={'stars': 'review_count'})
reviews_per_year.to_csv(r"D:\My-Projects\stonecap\data\processed\reviews\labelled_dataset\combined_reviews\reviews_per_year_per_business.csv", index=False)

In [ ]:
review_count_pivot_table = reviews.pivot_table(values='stars', index='business_id', aggfunc={'stars': ['mean', 'count']}).reset_index().sort_values('count')
review_count_pivot_table.to_csv(r"D:\My-Projects\stonecap\data\processed\business\reviews_star_pivot.csv")

In [10]:
reviews = reviews.drop('text', axis=1)

reviews.to_csv(r"D:\My-Projects\stonecap\data\processed\reviews\labelled_dataset\combined_reviews\clean_combined_reviews.csv", index=False)

# Business

In [68]:
business = pd.read_csv(r"D:\My-Projects\stonecap\data\processed\business\business.csv")
business.rename(columns={'stars': 'avg_stars'}, inplace=True)
business['country'] = 'US'

In [69]:
business.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32855 entries, 0 to 32854
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   business_id                 32855 non-null  object 
 1   name                        32855 non-null  object 
 2   city                        32855 non-null  object 
 3   state                       32855 non-null  object 
 4   postal_code                 32855 non-null  int64  
 5   latitude                    32855 non-null  float64
 6   longitude                   32855 non-null  float64
 7   avg_stars                   32855 non-null  float64
 8   review_count                32855 non-null  int64  
 9   Monday                      32855 non-null  object 
 10  Tuesday                     32855 non-null  object 
 11  Wednesday                   32855 non-null  object 
 12  Thursday                    32855 non-null  object 
 13  Friday                      328

In [70]:
business.to_csv(r"D:\My-Projects\stonecap\data\processed\business\clean_business.csv")

In [71]:
# Merging ambience labels together
ambience_merge_dict = {
    'absent': 'absent',
    'casual': 'casual',
    'classy': 'upscale',
    'divey': 'casual',
    'hipster': 'trendy',
    'intimate': 'upscale',
    'romantic': 'romantic',
    'touristy': 'romantic',
    'trendy': 'trendy',
    'upscale': 'upscale'
}

business.loc[business['Ambience'] == '']
business.loc[:, 'Ambience'] = business['Ambience'].map(ambience_merge_dict)

# merging alcohol labels
alcohol_merge_dict = {'none': False,
                      'full_bar': True,
                      'beer_and_wine': True}

business.loc[:, 'Alcohol'] = business['Alcohol'].map(alcohol_merge_dict)

business.loc[:, 'Alcohol'] = business['Alcohol'].fillna(False)


business_attr_redundant = ['RestaurantsGoodForGroups', 
'Caters'                 ,
'OutdoorSeating'         ,
'BusinessAcceptsCreditCards',
'BikeParking'               ,
'RestaurantsReservations'   ,
'BusinessParking'           ,
'WiFi'                      ,
'HasTV'                     ,
'GoodForKids'               ,
'WheelchairAccessible']  

business = business.drop(business_attr_redundant, axis=1)

C:\Users\blahb\AppData\Local\Temp\ipykernel_14244\2331710794.py:25: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  business.loc[:, 'Alcohol'] = business['Alcohol'].fillna(False)


In [73]:
business.to_csv(r"D:\My-Projects\stonecap\data\processed\business\clean_business_2.csv")